In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import DataLoader
from thermography_dataset_one_layer import ThermDataset
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
args = {'lr':0.01,
        'epochs':100,
        'noise':0.001,
        'train size':0.7,
        'spec scale':10**12
        }

In [3]:
def df_to_tensor(df):
    return torch.tensor(df.values, dtype=torch.float32)

# Prepare Data

In [4]:
df = pd.read_excel('combined_data.xlsx')

In [5]:
X = df.iloc[:,11:]
y = df.iloc[:,:11]

In [6]:
X = X.apply(lambda x: x*args['spec scale'])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=args['train size'], random_state=101)

# Design Models

In [8]:
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.lin1 = nn.Linear(input_size, 45)
        self.lin2 = nn.Linear(45, 60)
        self.lin3 = nn.Linear(60, 75)
        self.lin4 = nn.Linear(75, 60)
        self.lin_fin = nn.Linear(60, output_size)


    def forward(self, x):
        x = F.leaky_relu(self.lin1(x))
        x = F.leaky_relu(self.lin2(x))
        x = F.leaky_relu(self.lin3(x))
        x = F.leaky_relu(self.lin4(x))
        x = self.lin_fin(x)
        return x

In [9]:
n_samples = X_train
input_size = 66
output_size = 1

# store models in descending order (11, 10, 9...)
models = []
for i in range(11):
    models.append(Net(input_size+i, output_size))

# Training Loops

In [10]:
learning_rate = args['lr']
criterion = nn.MSELoss()

In [11]:
for i in range(11):
    best_loss = np.inf
    best_epoch = 0

    print(f'Layer {11-i}')

    optimizer = torch.optim.Adam(models[i].parameters(), lr=learning_rate)
    model = models[i]

    for epoch in range(args['epochs']):
        # empty gradients
        optimizer.zero_grad()

        # get tensors
        inputs = df_to_tensor(X_train)
        outputs = df_to_tensor(y_train.iloc[:,10-i]).reshape(-1,1)

        # forward pass
        pred = model(inputs)

        print(pred)
        print(outputs)
        # loss
        loss = criterion(pred, y_train)

        # update
        optimizer.step()

        if (epoch+1) % int(args['epochs']) == 0:
            print(f'epoch: {epoch+1}, loss = {loss}')

        if loss < best_loss:
            best_loss = loss
            best_epoch = epoch + 1

    print(f'best loss: {best_loss} in epoch {best_epoch}\n')


Layer 11
tensor([[0.1289],
        [0.1101],
        [0.1211],
        ...,
        [0.1209],
        [0.0921],
        [0.1117]], grad_fn=<AddmmBackward0>)
tensor([[369.5152],
        [352.1255],
        [363.4406],
        ...,
        [362.3184],
        [301.3779],
        [353.5918]])


TypeError: 'numpy.int32' object is not callable